In [ ]:
%pip install scikit-learn

In [ ]:
# importation des datasets
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer

# à enlever car la target de diabetes n'est pas bien définie
#from sklearn.datasets import load_diabetes

In [ ]:
from jmetal.core.problem import BinaryProblem
from jmetal.core.solution import BinarySolution
# import des librairie metals
from jmetal.algorithm.singleobjective.local_search \
import LocalSearch
from jmetal.operator import BitFlipMutation

#from jmetal.util.observer import ProgressBarObserver
from jmetal.util.observer import PrintObjectivesObserver
from jmetal.util.termination_criterion \
import StoppingByEvaluations

# Importez le problème OneMax inclu
from jmetal.problem import OneMax
import pandas as pd
import time as tm
import random

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [60]:
class SA(OneMax):
    def __init__(self, dataset: pd.DataFrame, classification_algorithm: str, select_features: bool):
        super(OneMax, self).__init__()
        self.number_of_bits = dataset.data.shape[1]
        self.number_of_objectives = 1
        self.number_of_variables = 1
        self.number_of_constraints = 0
        self.dataset = dataset
        self.obj_directions = [self.MINIMIZE]
        self.obj_labels = ['SA']
        self.classification_algorithm = classification_algorithm
        self.select_features = select_features

    def create_solution(self) -> BinarySolution:
      """
      Crée une solution binaire aléatoire
      On va prendre un vecteur de taille number_of_bits
      et chaque élément du vecteur sera soit 0 soit 1
      à 0 ça veut dire que l'attribut correspondant n'est pas sélectionné
      et à 1 ça veut dire que l'attribut correspondant est sélectionné
      """
      new_solution = BinarySolution(number_of_variables=self.number_of_variables,
                                    number_of_objectives=self.number_of_objectives,
                                    number_of_constraints=self.number_of_constraints)
      if self.select_features:
        new_solution.variables[0] = [random.randint(0, 1) for _ in range(self.number_of_bits)]
        random_idx = random.randint(0, self.number_of_bits-1)
        new_solution.variables[0][random_idx] = 1
      else:
        new_solution.variables[0] = [1 for _ in range(self.number_of_bits)]
      return new_solution

  
    def evaluate(self, solution: BinarySolution) -> BinarySolution:
        """
        Évalue la solution avec les algos knn et renvoie un score par rapport 
        au score de classification de l'algo knn
        """
        # On récupère les attributs sélectionnés
        selected_features = [i for i in range(self.number_of_bits) if solution.variables[0][i] == 1]
        # On récupère les colonnes correspondantes dans le dataset
        df = pd.DataFrame(self.dataset.data, columns=self.dataset.feature_names)
        df['target'] = self.dataset.target
        x = df.iloc[:, selected_features]
        y = df.iloc[:, df.columns.get_loc('target')] 
        # On divise le dataset en ensemble d'entrainement et ensemble de test   
        # 70% pour l'entrainement et 30% pour le test
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
        # On entraine le modèle
        if self.classification_algorithm == 'KNN':
            model = KNeighborsClassifier(n_neighbors=self.number_of_bits+1)
        elif self.classification_algorithm == 'SVM':
            model = SVC()
        model.fit(x_train, y_train)
        # On prédit les résultats
        y_pred = model.predict(x_test)
        # On renvoie le score de classification
        solution.objectives[0] = model.score(x_test, y_test)
        return solution


    def get_name(self) -> str:
        return 'SA'
        

In [61]:
# il faudra faire un svm et knn sans sa et voir la différence


from jmetal.algorithm.singleobjective.genetic_algorithm \
import GeneticAlgorithm
from jmetal.operator import BitFlipMutation, SPXCrossover
from jmetal.problem import OneMax
from jmetal.util.termination_criterion import StoppingByEvaluations


def run_genetic_algorithm(population_size, offspring_population_size, 
                          mutation_rate, crossover_rate, dataset, classification_algorithm,
                          select_features,
                          max_evaluations=5000):
    """
    Runs a genetic algorithm to solve the SA problem.

    Args:
        population_size (int): The size of the initial population.
        offspring_population_size (int): The size of the offspring 
        population.
        mutation_rate (float): The probability of mutation for each 
        bit in the population.
        crossover_rate (float): The probability of crossover between two 
        individuals in the population.
        max_evaluations (int, optional): The maximum number of evaluations allowed. 
        Defaults to 5000.
        dataset (pd.DataFrame): The dataset to use for the SA problem.

    Returns:
        int: The fitness value of the best solution found by the genetic algorithm.
    """

    problem = SA(dataset, classification_algorithm, select_features)
    algorithm = GeneticAlgorithm(
        problem=problem,
        population_size=population_size,
        offspring_population_size=offspring_population_size,
        mutation=BitFlipMutation(mutation_rate / problem.number_of_bits),
        crossover=SPXCrossover(crossover_rate),
        termination_criterion=StoppingByEvaluations(max_evaluations=max_evaluations),
    )


    algorithm.observable.register(observer=PrintObjectivesObserver(100))

    # Exécutez l'algorithme de recherche locale
    algorithm.run()
    result = algorithm.get_result()

    print("Algorithm: Genetic Algorithm")  
    print("Problem SA")
    print("Solution: " + result.get_binary_string())
    print("Fitness:  " + str(result.objectives[0]))
    print("Computing time: " + str(algorithm.total_computing_time))

    return abs(int(result.objectives[0]))

    




In [57]:
def train(problem_function, problem_function_params, nb_runs=20):
  """
  Trains a problem function by running it multiple times 
  and collecting fitness results and running times.

  Parameters:
  problem_function (callable): The problem function to be trained.
  problem_function_params (dict): The parameters to be passed 
  to the problem function.
  nb_runs (int, optional): The number of times to run the problem function. 
  Defaults to 20.

  Returns:
  pd.Series: A series containing the fitness results of each run.
  pd.Series: A series containing the running times of each run.
  """
  fitnesses = []
  running_times = []

  for _ in range(nb_runs):
    start_time = tm.time()
    fitness_result = problem_function(**problem_function_params)
    execution_time = tm.time() - start_time

    fitnesses.append(fitness_result)
    running_times.append(execution_time)

  return pd.Series(fitnesses), pd.Series(running_times)

In [64]:
dataset = load_iris()
classification_algorithm = 'SVM'
select_features = True
SAProblem_params_optimal_ag = {
         "population_size" : 10
        ,"offspring_population_size" : 10
        ,"mutation_rate" : 0.4
        ,"crossover_rate" : 1
        ,"dataset" : dataset
        ,"classification_algorithm" : classification_algorithm
        ,"select_features" : select_features
        , "max_evaluations":10
    }

fitnesses, running_times = train(problem_function=run_genetic_algorithm,
                                 problem_function_params=SAProblem_params_optimal_ag, nb_runs=5)

[2024-02-20 10:15:40,380] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:15:40,381] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:15:40,434] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:15:40,439] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:15:40,442] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:15:40,445] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:15:40,446] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:15:40,505] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:15:40,506] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:15:40,507] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:15:40,508] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:15:40,508] [jm

Algorithm: Genetic Algorithm
Problem SA
Solution: 0011
Fitness:  0.9555555555555556
Computing time: 0.06445121765136719
Algorithm: Genetic Algorithm
Problem SA
Solution: 0110
Fitness:  0.9777777777777777
Computing time: 0.06255698204040527
Algorithm: Genetic Algorithm
Problem SA
Solution: 0100
Fitness:  0.4444444444444444
Computing time: 0.04237008094787598
Algorithm: Genetic Algorithm
Problem SA
Solution: 0011
Fitness:  0.9555555555555556
Computing time: 0.03146719932556152
Algorithm: Genetic Algorithm
Problem SA
Solution: 0100
Fitness:  0.4666666666666667
Computing time: 0.03271317481994629


In [65]:
dataset = load_iris()
classification_algorithm = 'SVM'
select_features = False
SAProblem_params_optimal_ag = {
         "population_size" : 10
        ,"offspring_population_size" : 10
        ,"mutation_rate" : 0.4
        ,"crossover_rate" : 1
        ,"dataset" : dataset
        ,"classification_algorithm" : classification_algorithm
        ,"select_features" : select_features
        , "max_evaluations":10
    }

fitnesses, running_times = train(problem_function=run_genetic_algorithm,
                                 problem_function_params=SAProblem_params_optimal_ag, nb_runs=5)

[2024-02-20 10:16:17,534] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:16:17,535] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:16:17,586] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:16:17,587] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:16:17,587] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:16:17,588] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:16:17,588] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:16:17,622] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:16:17,622] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:16:17,622] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:16:17,623] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:16:17,623] [jm

Algorithm: Genetic Algorithm
Problem SA
Solution: 1111
Fitness:  1.0
Computing time: 0.0535428524017334
Algorithm: Genetic Algorithm
Problem SA
Solution: 1111
Fitness:  1.0
Computing time: 0.03523731231689453
Algorithm: Genetic Algorithm
Problem SA
Solution: 1111
Fitness:  1.0
Computing time: 0.04864788055419922
Algorithm: Genetic Algorithm
Problem SA
Solution: 1111
Fitness:  0.9555555555555556
Computing time: 0.06368899345397949
Algorithm: Genetic Algorithm
Problem SA
Solution: 1111
Fitness:  0.9111111111111111
Computing time: 0.03188896179199219


In [67]:
dataset = load_iris()
classification_algorithm = 'KNN'
select_features = True
SAProblem_params_optimal_ag = {
         "population_size" : 10
        ,"offspring_population_size" : 10
        ,"mutation_rate" : 0.4
        ,"crossover_rate" : 1
        ,"dataset" : dataset
        , "select_features" : select_features
        ,"classification_algorithm" : classification_algorithm
        , "max_evaluations":10
    }

fitnesses, running_times = train(problem_function=run_genetic_algorithm,
                                 problem_function_params=SAProblem_params_optimal_ag, nb_runs=5)

[2024-02-20 10:18:01,068] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:18:01,069] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:18:01,151] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:18:01,152] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:18:01,152] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:18:01,152] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:18:01,153] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:18:01,214] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:18:01,215] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:18:01,215] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:18:01,215] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:18:01,216] [jm

Algorithm: Genetic Algorithm
Problem SA
Solution: 1110
Fitness:  0.9333333333333333
Computing time: 0.08406591415405273
Algorithm: Genetic Algorithm
Problem SA
Solution: 0111
Fitness:  1.0
Computing time: 0.0629880428314209
Algorithm: Genetic Algorithm
Problem SA
Solution: 1111
Fitness:  0.9777777777777777
Computing time: 0.06194496154785156


[2024-02-20 10:18:01,372] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:18:01,373] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:18:01,373] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:18:01,374] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:18:01,374] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:18:01,445] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:18:01,446] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:18:01,446] [jmetal.core.algorithm] [DEBUG] Finished!


Algorithm: Genetic Algorithm
Problem SA
Solution: 0011
Fitness:  0.9777777777777777
Computing time: 0.09616994857788086
Algorithm: Genetic Algorithm
Problem SA
Solution: 0110
Fitness:  0.9333333333333333
Computing time: 0.07240009307861328


In [71]:
dataset = load_breast_cancer()
classification_algorithm = 'KNN'
select_features = True
SAProblem_params_optimal_ag = {
         "population_size" : 10
        ,"offspring_population_size" : 10
        ,"mutation_rate" : 0.4
        ,"crossover_rate" : 1
        ,"dataset" : dataset
        , "select_features" : select_features
        ,"classification_algorithm" : classification_algorithm
        , "max_evaluations":10
    }

fitnesses, running_times = train(problem_function=run_genetic_algorithm,
                                 problem_function_params=SAProblem_params_optimal_ag)

[2024-02-20 10:20:00,791] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:20:00,792] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...


[2024-02-20 10:20:00,955] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:20:00,956] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:20:00,957] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:20:00,957] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:20:00,958] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:20:01,107] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:20:01,108] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:20:01,109] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:20:01,109] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:20:01,109] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...


Algorithm: Genetic Algorithm
Problem SA
Solution: 010101000011011011010101001111
Fitness:  0.9122807017543859
Computing time: 0.16634297370910645
Algorithm: Genetic Algorithm
Problem SA
Solution: 011111101111101110111000000101
Fitness:  0.8713450292397661
Computing time: 0.15163588523864746


[2024-02-20 10:20:01,265] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:20:01,266] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:20:01,267] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:20:01,267] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:20:01,268] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:20:01,419] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:20:01,420] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:20:01,420] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:20:01,421] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:20:01,421] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...


Algorithm: Genetic Algorithm
Problem SA
Solution: 000000110000011101111110100100
Fitness:  0.9239766081871345
Computing time: 0.1580967903137207
Algorithm: Genetic Algorithm
Problem SA
Solution: 111000000000111100000101101110
Fitness:  0.9298245614035088
Computing time: 0.1534111499786377


...

Algorithm: Genetic Algorithm
Problem SA
Solution: 011010011011000001110100110100
Fitness:  0.9064327485380117
Computing time: 0.14949393272399902
Algorithm: Genetic Algorithm
Problem SA
Solution: 001001101001101011110001101010
Fitness:  0.9005847953216374
Computing time: 0.1658916473388672


[2024-02-20 10:20:03,697] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:20:03,697] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:20:03,698] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:20:03,698] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:20:03,699] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-02-20 10:20:03,850] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:20:03,851] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:20:03,851] [jmetal.core.algorithm] [DEBUG] Finished!
[2024-02-20 10:20:03,852] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-02-20 10:20:03,853] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...


Algorithm: Genetic Algorithm
Problem SA
Solution: 100001000001000100100001001011
Fitness:  0.935672514619883
Computing time: 0.14937901496887207
Algorithm: Genetic Algorithm
Problem SA
Solution: 111111011111010000000110110000
Fitness:  0.8888888888888888
Computing time: 0.15352797508239746


[2024-02-20 10:20:04,000] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-02-20 10:20:04,000] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-02-20 10:20:04,001] [jmetal.core.algorithm] [DEBUG] Finished!


Algorithm: Genetic Algorithm
Problem SA
Solution: 010010010111000111111101000110
Fitness:  0.935672514619883
Computing time: 0.14911580085754395
